# DLC Postprocess Library

Run after training has been performed, and at least 1 video has been analyzed

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import json
import os

from PyQt5.QtWidgets import QFileDialog
from IPython.display import display, HTML

# Append base directory
import os,sys,inspect
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parpath = os.path.dirname(thispath)
sys.path.append(parpath)
print("Appended script directory", parpath)

from lib.parse_dlc_results import parse_dlc_csv
from lib.constraints import compute_constraints
from lib.plots import plotStatistics
from lib.stickman import stickman
from lib.qt_wrapper import gui_fname, gui_fpath
from lib.sampling import selectUniform
from lib.hdf5_wrapper import npStrArr2h5

In [ ]:
%load_ext autoreload
%autoreload 2

from lib.constraints import likelihood_constrain, velocity_constrain, edge_constrain

### Enter parameters for the analysis of DLC marking

1. **Video File** - The raw video that has been tracked by DLC
2. **Tracking File** - The .CSV file where DLC has put tracking information (usually in the same folder as the raw video after tracking has been done. **Take care to select matching video and tracking file**)
3. **Postprocess Template** - Located in dlc-scripts/scripts/postprocess-templates/. Make one or more templates manually for your individual setup. Template specifies what tracking points will be discarded in the final result, based on: DLC likelihood estimate, velocity of the points (in pixel) and relative edge lengths of edges, defined in the same file.
* **Result Path** - Place where stickman video and filtered tracking data will be written.

In [ ]:
pwd_remote="./"
#pwd_remote = "/run/user/1000/gvfs/smb-share:server=130.60.51.15,share=neurophysiology-storage2/Sipila/aaaPDDATA/ALLDATA/TRAININGvideos/"
param = {}
#param["CONF_FNAME"] = gui_fname("Select config file...", "./", "Config Files (*.yaml)")
param["AVI_FNAME"] = gui_fname("Select original video file...", pwd_remote, "Video Files (*.avi *.mp4)")
tmp_pwd = os.path.dirname(param["AVI_FNAME"])
param["CSV_FNAME"] = gui_fname("Select tracking file...", tmp_pwd, "CSV Files (*.csv)")
param["TMP_FNAME"] = gui_fname("Select Postprocess Template Filename...", "./", "Template Files (*.json)")
param["REZ_FPATH"] = gui_fpath("Select result path", "./")

#print("Using config file", param["CONF_FNAME"])
print("Using original video", param["AVI_FNAME"])
print("Using tracking file", param["CSV_FNAME"])
print("Using template file", param["TMP_FNAME"])
print("Results will be saved in", param["REZ_FPATH"])

In [ ]:
#########################
# Parse CSV file
#########################
NODE_NAMES_TRACKED, X, Y, P = parse_dlc_csv(param["CSV_FNAME"])
nFrames, nNodes = X.shape
print("Movie has", nFrames, "frames and", nNodes, "nodes")

#########################
# Parse JSON file
#########################
with open(param["TMP_FNAME"]) as json_file:
    param.update(json.load(json_file))
    
#########################
# Test if template node count match the tracked
#########################
if nNodes != len(param["NODE_NAMES"]):
    raise ValueError("The tracked nodes", NODE_NAMES_TRACKED, "are not consistent with those given in the template file, namely ", param["NODE_NAMES"])
else:
    print("The tracked node names were", NODE_NAMES_TRACKED)
    print("The template node names will be used (order matters):", param["NODE_NAMES"])

### Analyse data, compute and plot results

Determine frames, that
* Have low confidence based on DLC self-reported analysis
* Do not fulfill node constraints (e.g. excessive velocity)
* Do not fulfill edge constraints (e.g. too large or too small edges)

In [ ]:
%matplotlib notebook
    
# Compute Constraints
constr_dict = compute_constraints(X, Y, P, param)
display(constr_dict["summary"])

# Plot Statistics
plotStatistics(param, constr_dict)

### Create stickman video and write it to file

In [ ]:
#########################
# Make Stickman Video
#########################
stickman(X, Y, param, constr_dict)